# Repeating previous experiments using networks with sparse structure

Strategy: use the code you have already. (arch = mlp_noreluout_lastlayernottrainable)
- Initialize weights 1 to have the correct sparsity structure
- Initialize weights 2 to 1
- Use a special type of optimizer to only update non-zero entries

Some properties of the experiments:
- Only looking at 3-layer networks
- Layer2-to-layer3 weights are fixed to equal 1 (not trainable)
- Generated inputs are iid

These experiments are akin to previous experiments: looking to see how often we get good performance (0 loss) for different network structures. In particular, we are interested in the number of global and local optima, which we evaluate using the final loss and the final gap between estimated and true weights.

The first set of experiments involve generating true labels using the same network used to learn those weights. The second set of experiments involve generating true labels using a fully-connected network while learning with a sparser structure.

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function
import tensorflow as tf
import numpy as np
import os,time,seaborn
import matplotlib.pyplot as plt
import arch,dl_utils,utils

np.set_printoptions(precision=2,suppress=True)
gpu_id = 1

%matplotlib inline

maindir = './temp_sparse/'
q = 0

In [2]:
# Same as graph_builder_wrapper except for sparser structures
# CURRENTLY ONLY WORKS WITH mlp_noreluout_lastlayernottrainable
# (M is a mask for only the weights from input to hidden 1 layers)
def graph_builder_wrapper_sparse(input_dict,M,build_func=arch.mlp,lr_initial=0.01,max_save=100):
    graph = build_func(input_dict)
    
    # Loss
    y = tf.placeholder(tf.float32, shape=[None,1])
    total_loss = dl_utils.loss(y, graph['yhat'])
    
    # W gap
    w = {k:tf.placeholder(tf.float32, shape=[None,None]) for k in graph if 'weights' in k}
    total_w_gap = dl_utils.w_gap(w, {k:graph[k] for k in graph if 'weights' in k})
    
    # Optimizer
    learning_rate = tf.Variable(lr_initial, name='learning_rate')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    grads = optimizer.compute_gradients(total_loss)
    modified_grads = []
    for gv in grads:
        if gv[0] is None: modified_grads.append(gv)
        else: modified_grads.append((tf.multiply(gv[0], M), gv[1]))
    opt_step = optimizer.apply_gradients(modified_grads)
    
    graph['y'] = y
    graph['w'] = w
    graph['opt_step'] = opt_step
    graph['total_loss'] = total_loss
    graph['total_w_gap'] = total_w_gap
    graph['learning_rate'] = learning_rate
    graph['saver'] = tf.train.Saver(max_to_keep=100)
    return graph

def generate_two_plots(all_final_loss,all_final_w_gaps,all_initial_w_gaps):
    f, (ax1,ax2) = plt.subplots(1,2,figsize=(16,6))
    seaborn.regplot(all_initial_w_gaps,all_final_loss,ax=ax1)
    ax1.set(xlabel='initial w gap', ylabel='final loss')
    seaborn.regplot(all_final_w_gaps,all_final_loss,ax=ax2)
    ax2.set(xlabel='final w gap', ylabel='final loss')

# Experiment set 1: true labels generated with sparse architectures

### Sparse structure 1

Each hidden neuron goes to one input

In [3]:
# Sparse structure 1: each hidden neuron goes to one input
def generate_sparse_w_1(p,r,seed=0):
    np.random.seed(seed)
    def sparse_mask_1(p,r): return np.repeat(np.eye(p),r,axis=0)
    M = sparse_mask_1(p,r)
    Z = np.random.normal(0,1,[p*r,p]) # std is 1 since each hidden neuron is connected to 1 input
    W1 = (Z*M).T
    W2 = np.ones([h,1])
    return {'weights1':W1,'weights2':W2},M.T

In [ ]:
num_sims = 100 # number of simulations
N = 10000      # number of samples

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable
p = 5    # number of inputs

list_r = range(1,5)
outputs = {r:[] for r in list_r}
start = time.time()

# r = number of neurons each input is connected to
for i,r in enumerate(list_r):
    
    for t in range(num_sims):
        
        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)
        
        seed = (r-1)*num_sims+t
        savedir = '%ssparse_structure_1/initialization_%s_r%s/'%(maindir,t,r)
    
        # Generate data
        w_true,M = generate_sparse_w_1(p,r,seed=seed)
        X = dl_utils.generate_X(N,0,input_dict,cov_is_eye=True,seed=seed)
        Y = dl_utils.generate_output(X,w_true,input_dict,build_func=build_func)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_1(p,r,seed=seed+1)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)
        
#         # If weights exist already, load useful information
#         _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
#                                                                     savedir,build_func)
#         initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
#         out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]
        
        # Save useful information
        outputs[r].append(out)
        
        print('\rr = %s, %s/%s simulations done (%.2f s elapsed)'\
              %(r,t+1,num_sims,time.time()-start),end='')
        
pickle.dump(outputs,file('%ssparse_structure_1/outputs.pickle'%(maindir),'wb'))

Epoch 3/100 (0.506 s), batch 11.0/100 (0.010 s): loss 0.253, w gap: 0.635

Epoch 5/100 (0.707 s), batch 38.0/100 (0.038 s): loss 0.010, w gap: 0.031

Epoch 7/100 (0.909 s), batch 58.0/100 (0.052 s): loss 0.001, w gap: 0.002

Epoch 9/100 (1.109 s), batch 83.0/100 (0.074 s): loss 0.000, w gap: 0.000

Epoch 11/100 (1.310 s), batch 93.0/100 (0.075 s): loss 0.000, w gap: 0.000

...

Epoch 93/100 (8.963 s), batch 89.0/100 (0.088 s): loss 0.442, w gap: 13.983

Epoch 95/100 (9.162 s), batch 100.0/100 (0.096 s): loss 0.905, w gap: 13.977

Epoch 98/100 (9.368 s), batch 1.0/100 (0.001 s): loss 0.668, w gap: 13.958

Epoch 100/100 (9.570 s), batch 20.0/100 (0.017 s): loss 0.658, w gap: 13.922

Epoch 100/100 (9.639 s), batch 100.0/100 (0.087 s): loss 0.853, w gap: 13.941
Epoch 100/100 (9.579 s), batch 100.0/100 (0.089 s): loss 0.141, w gap: 5.157
Epoch 100/100 (9.793 s), batch 100.0/100 (0.103 s): loss 0.004, w gap: 0.296
Epoch 100/100 (9.728 s), batch 100.0/100 (0.105 s): loss 0.000, w gap: 1.951
Epoch 100/100 (9.627 s), batch 100.0/100 (0.108 s): loss 0.000, w gap: 5.925
Epoch 100/100 (9.621 s), batch 100.0/100 (0.087 s): loss 0.000, w gap: 5.855
Epoch 100/100 (9.672 s), batch 100.0/100 (0.100 s): loss 0.000, w gap: 8.259
Epoch 100/100 (9.641 s), batch 100.0/100 (0.093 s): loss 0.000, w gap: 2.175
Epoch 100/100 (9.687 s), batch 100.0/100 (0.083 s): loss 0.000, w gap: 2.271
Epoch 100/100 (9.581 s), batch 100.0/100 (0.084 s): loss 0.069, w gap: 5.505
Epoch 100/100 (9.694 s), batch 100.0/100 (0.097 s): loss 0.000, w gap: 12.566
Epoch 100/100 (9.592 s), batch 100.0/100 (0.107 s): loss 0.000, w gap: 35.774
Epoch 100/100 (9.665 s), batch 100.0/100 (0.093 s): loss 0.000, w gap: 5.

### Sparse structure 2

Each hidden neuron goes to s random inputs

In [ ]:
# Sparse structure 2: each hidden neuron goes to s random inputs
def generate_sparse_w_2(p,s,h,seed=0,maskseed=0):
    def sparse_mask_2(p,s,h,seed=0):
        np.random.seed(seed)
        M = np.zeros((h,p))
        for i in range(h):
            M[i,np.random.choice(range(p),s,replace=False)] = 1
        return M
    M = sparse_mask_2(p,s,h,seed=maskseed)
    np.random.seed(seed)
    Z = np.random.normal(0,1./np.sqrt(s),[h,p])
    W1 = (Z*M).T
    W2 = np.ones([h,1])
    return {'weights1':W1,'weights2':W2},M.T

In [ ]:
# Case where h > p

num_sims = 100 # number of simulations
N = 10000      # number of samples

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable
p = 5    # number of inputs
h = 10   # number of hidden nodes

list_s = range(1,5)
outputs = {s:[] for s in list_s}
start = time.time()

# r = number of neurons each input is connected to
for i,s in enumerate(list_s):
    
    for t in range(num_sims):
        
        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)
        
        seed = (s-1)*num_sims+t
        savedir = '%ssparse_structure_2/initialization_%s_s%s/'%(maindir,t,s)
    
        # Generate data
        w_true,M = generate_sparse_w_2(p,s,h,seed=seed,maskseed=seed)
        X = dl_utils.generate_X(N,0,input_dict,cov_is_eye=True,seed=seed)
        Y = dl_utils.generate_output(X,w_true,input_dict,build_func=build_func)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_2(p,s,h,seed=seed+1,maskseed=seed)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)
        
#         # If weights exist already, load useful information
#         _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
#                                                                     savedir,build_func)
#         initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
#         out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]
        
        # Save useful information
        outputs[r].append(out)
        
        print('\rr = %s, %s/%s simulations done (%.2f s elapsed)'\
              %(r,t+1,num_sims,time.time()-start),end='')
        
pickle.dump(outputs,file('%ssparse_structure_2/outputs.pickle'%(maindir),'wb'))

In [ ]:
# Case where h < p

num_sims = 100 # number of simulations
N = 10000      # number of samples

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable
p = 5    # number of inputs
h = 3    # number of hidden nodes

list_s = range(1,5)
outputs = {s:[] for s in list_s}
start = time.time()

# r = number of neurons each input is connected to
for i,s in enumerate(list_s):
    
    for t in range(num_sims):
        
        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)
        
        seed = (s-1)*num_sims+t
        savedir = '%ssparse_structure_2/initialization_%s_s%s/'%(maindir,t,s)
    
        # Generate data
        w_true,M = generate_sparse_w_2(p,s,h,seed=seed,maskseed=seed)
        X = dl_utils.generate_X(N,0,input_dict,cov_is_eye=True,seed=seed)
        Y = dl_utils.generate_output(X,w_true,input_dict,build_func=build_func)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_2(p,s,h,seed=seed+1,maskseed=seed)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)
        
#         # If weights exist already, load useful information
#         _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
#                                                                     savedir,build_func)
#         initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
#         out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]
        
        # Save useful information
        outputs[r].append(out)
        
        print('\rr = %s, %s/%s simulations done (%.2f s elapsed)'\
              %(r,t+1,num_sims,time.time()-start),end='')
        
pickle.dump(outputs,file('%ssparse_structure_2/outputs.pickle'%(maindir),'wb'))

# Experiment set 2: true labels generated with dense architectures

Layer2-layer3 weights all set to 1 still.

In [ ]:
# Generate labels using dense networks

np.random.seed(0)
p = 10
h = 3
w = {'weights1':np.random.normal(0,1./np.sqrt(p),[p,h]),
     'weights2':np.ones([h,1])}

N = 10000
X = dl_utils.generate_X(N,0,input_dict,cov_is_eye=True,seed=seed)
Y = dl_utils.generate_output(X,w_true,input_dict,build_func=arch.mlp_noreluout_lastlayernottrainable)

### Sparse structure 1

In [ ]:
num_attempts = 10 # different initializations may have different performance, so try a few

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable

list_r = range(1,5)
outputs = {r:[] for r in list_r}
start = time.time()

# r = number of neurons each input is connected to
for i,r in enumerate(list_r):
    
    for t in range(num_attempts):

        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)

        seed = (r-1)*num_sims+t
        savedir = '%ssparse_structure_1_denselabels/initialization_r%s/'%(maindir,r)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_1(p,r,seed=seed)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)

    #     # If weights exist already, load useful information
    #     _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
    #                                                                 savedir,build_func)
    #     initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
    #     out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]

        # Save useful information
        outputs[r].append(out)

pickle.dump(outputs,file('%ssparse_structure_1_denselabels/outputs.pickle'%(maindir),'wb'))

### Sparse structure 2

In [ ]:
# Case where h < p

num_attempts = 10 # different initializations may have different performance, so try a few

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable

h = 3    # number of hidden nodes

list_s = range(1,9)
outputs = {s:[] for s in list_s}
start = time.time()

# r = number of neurons each input is connected to
for i,s in enumerate(list_s):
    
    for t in range(num_attempts):

        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)

        seed = (r-1)*num_sims+t
        savedir = '%ssparse_structure_2_denselabels_pgh/initialization_r%s/'%(maindir,r)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_2(p,s,h,seed=seed+1,maskseed=seed)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)

    #     # If weights exist already, load useful information
    #     _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
    #                                                                 savedir,build_func)
    #     initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
    #     out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]

        # Save useful information
        outputs[r].append(out)

pickle.dump(outputs,file('%ssparse_structure_2_denselabels_pgh/outputs.pickle'%(maindir),'wb'))

In [ ]:
# Case where h > p

num_attempts = 10 # different initializations may have different performance, so try a few

# Network parameters
num_epochs = 100
batch_size = 100
build_func = arch.mlp_noreluout_lastlayernottrainable

h = 15    # number of hidden nodes

list_s = range(1,9)
outputs = {s:[] for s in list_s}
start = time.time()

# r = number of neurons each input is connected to
for i,s in enumerate(list_s):
    
    for t in range(num_attempts):

        h = p*r  # number of neurons
        input_dict = dict(p=p,h=h)

        seed = (r-1)*num_sims+t
        savedir = '%ssparse_structure_2_denselabels_plh/initialization_r%s/'%(maindir,r)

        # initialize weights to have desired sparsity structure
        w_init,M = generate_sparse_w_2(p,s,h,seed=seed+1,maskseed=seed)

        # Build graph and train..
        tf.reset_default_graph()
        with tf.device("/gpu:%s"%(gpu_id)):
            graph = graph_builder_wrapper_sparse(input_dict,M,build_func=build_func)
            out = dl_utils.train(X,Y,graph,num_epochs,batch_size,w_true,w_init,savedir=savedir)

    #     # If weights exist already, load useful information
    #     _,final_train_loss,final_w_gap = dl_utils.get_train_out(X,Y,w_true,input_dict,
    #                                                                 savedir,build_func)
    #     initial_w_gap = dl_utils.compute_w_gap(w_true,w_init)
    #     out = ['','','','',initial_w_gap,final_train_loss,final_w_gap]

        # Save useful information
        outputs[r].append(out)

pickle.dump(outputs,file('%ssparse_structure_2_denselabels_plh/outputs.pickle'%(maindir),'wb'))